# Load packages

In [1]:
import pandas as pd
import json
from glob import glob
import re
from Levenshtein import distance as levenshtein_distance
from itertools import product

# Clean data (export wide form data for CDER group)

In [2]:
files = [f for f in glob('C:/Users/Cole/Documents/DATA/Network_DATA/Fall2020_MID/*') if 'wide' not in f and 'edgelist' not in f]
for f in files:
    df = pd.read_csv(f)
    df.question_content = df.question_content.apply(json.loads)
    df.answers = df.answers.apply(json.loads)

    ids = df.user_id.unique()
    student_data = []
    for id in ids:
        df_one = df.loc[df.user_id == id, :].reset_index(drop = True)
        wide_data = {'user_id':df_one.loc[0, 'user_id'], 'name':df_one.loc[0, 'name']}
        for r, data in df_one.iterrows():
            questions = data['question_content']
            questions = [q['value'] for q in questions if q['type'] == 'text']

            answers = data['answers']
            answers = [answers[a] for a in answers.keys()] 
            wide_data.update(dict(zip(questions, answers)))

        student_data.append(wide_data)
    pd.DataFrame.from_dict(student_data).to_csv(f[:-4] + '_wide.csv', index = False)

# Load and clean network data function

In [26]:
def CreateEdgelist(file, cols, roster_file = None, csv_skip = None, merge_name = False, first_name_col = None, last_name_col = None):
    df = pd.read_csv(file, usecols = cols, skiprows = csv_skip)[cols]
    cols2 = cols.copy()
    if merge_name:
        df['name'] = df[first_name_col].fillna('') + ' ' + df[last_name_col].fillna('')
        cols2 = [[cols[0]], ['name'], cols[-4:]]
        cols2 = [c for sublist in cols2 for c in sublist]
        df = df[cols2]
    df = df.rename(columns = dict(zip(cols2, ['user_id', 'name', 'strong_lect_disc', 'meaning_other', 'strong_lab', 
                                              'meaning_lab']))).fillna('')
    df.name = df.name.str.lower()
    df.strong_lect_disc = df.strong_lect_disc.str.lower()
    df.meaning_other = df.meaning_other.str.lower()
    df.strong_lab = df.strong_lab.str.lower()
    df.meaning_lab = df.meaning_lab.str.lower()

    if roster_file is not None:
        df_reference = pd.read_csv(roster_file, 
                                   usecols = ['First Name', 
                                              'Last Name'])[['First Name', 
                                                             'Last Name']].rename(columns = {'First Name':'first_name', 
                                                                                             'Last Name':'last_name'})
        df_reference.first_name = df_reference.first_name.str.lower()
        df_reference.last_name = df_reference.last_name.str.lower()
        df_reference['name'] = df_reference.first_name + ' ' + df_reference.last_name
    else:
        df_reference = df.copy()
        df_reference['first_name'] = df_reference.name.str.split(' ', n = 1, expand = True)[0]
        df_reference['last_name'] = df_reference.name.str.split(' ').str[-1]
    
    df_reference['full_name'] = df_reference.name.str.replace(' ', '')
    df_reference['duplicated_first'] = df_reference.duplicated(subset = 'first_name', keep = False)
    df_reference['duplicated_last'] = df_reference.duplicated(subset = 'last_name', keep = False)
    df_reference.loc[df_reference.duplicated_first == True, 'first_name'] = ''
    df_reference.loc[df_reference.duplicated_last == True, 'last_name'] = ''
    df_reference = df_reference.drop(columns = ['duplicated_first', 'duplicated_last'])

    dfs = []
    for attr in ['strong_lect_disc', 'meaning_other', 'strong_lab', 'meaning_lab']:
        connections = []
        for r, data in df.iterrows():
            sentence = data[attr]
            tokens = re.split(' |,|\n|;|\.', sentence)
            tokens = [t for t in tokens if t != '']

            first_names = [n for n in list(df_reference.first_name) if n != '']
            student_connections = [pair[1] for pair in list(product(tokens, first_names)) if levenshtein_distance(pair[0], 
                                                                                                                  pair[1]) == 0]
            if student_connections:
                connections.append(list(product([data['name']], 
                                                df_reference.loc[df_reference.first_name.isin(student_connections), 
                                                                 'name'].tolist())))

            last_names = [n for n in list(df_reference.last_name) if n != '']
            student_connections = [pair[1] for pair in list(product(tokens, last_names)) if levenshtein_distance(pair[0], 
                                                                                                                 pair[1]) == 0]
            if student_connections:
                connections.append(list(product([data['name']], 
                                                df_reference.loc[df_reference.last_name.isin(student_connections), 
                                                                 'name'].tolist())))

            paired_tokens = [''.join(pair) for pair in zip(tokens[:-1], tokens[1:])]
            full_names = list(df_reference.full_name)
            student_connections = [pair[1] for pair in list(product(paired_tokens, 
                                                                    full_names)) if levenshtein_distance(pair[0], pair[1]) <= 
                                   0.3 * len(pair[1])]

            if student_connections:
                connections.append(list(product([data['name']], 
                                                df_reference.loc[df_reference.full_name.isin(student_connections),
                                                                 'name'].tolist())))

        edgelist = pd.DataFrame(sorted(list(set([c for student in connections for c in student]))), columns = ['From', 'To'])
        edgelist['attr'] = attr
        dfs.append(edgelist)
    df_out = pd.concat(dfs, axis = 0).reset_index(drop = True)
    df_out = df_out[(df_out['From'].map(len) > 1) & (df_out['To'].map(len) > 1)]
    df_out.to_csv(file[:-8] + 'edgelist.csv', index = False)

    df_out[['From', 'To']] = df_out[['From', 'To']].stack().rank(method = 'dense').astype(int).unstack()
    df_out.to_csv(file[:-8] + 'edgelist_anon.csv', index = False)

In [8]:
cols_Fall2020MID = ['user_id', 'name', 
                    'Please list any students in this physics class that you think are particularly strong in the lecture/discussion section material.', 
                    'Please list any students in this physics class that you had a meaningful interaction* with about other aspects of the course this week.',
                    'Please list any students in this physics class that you think are particularly strong in the lab material.',
                    'Please list any students in this physics class that you had a meaningful interaction* with about lab material this week.']
for f in glob('C:/Users/Cole/Documents/DATA/Network_DATA/Fall2020_MID/*wide.csv'):
    CreateEdgelist(f, cols_Fall2020MID)

In [30]:
cols_Fall2020POST1112 = ['Q5a', 'Q5b', 'Q5c', 'Q143', 'Q141', 'Q142', 'Q139']
CreateEdgelist('C:/Users/Cole/Documents/DATA/Network_DATA/Fall2020_POST/Fall2020_Cornell_University_Phys_1112_Holmes_POST_R_3QW82bxsvpr9AiT_January+10,+2021_17.39.csv', 
               cols_Fall2020POST1112, 
               roster_file = 'C:/Users/Cole/Documents/DATA/Network_DATA/Rosters/PHYS1112_Fall_2020_roster.csv', 
               csv_skip = [1, 2], merge_name = True, first_name_col = 'Q5c', last_name_col = 'Q5b')

cols_Fall2020POST1116_2218 = ['Q5a', 'Q5b', 'Q5c', 'Q222.1', 'Q218', 'Q220.1', 'Q216']
CreateEdgelist('C:/Users/Cole/Documents/DATA/Network_DATA/Fall2020_POST/Fall2020_Cornell_University_Phys_1116_Holmes_POST_R_2SxhUhMaXEhvmoi_January+10,+2021_17.39.csv', 
               cols_Fall2020POST1116_2218, 
               roster_file = 'C:/Users/Cole/Documents/DATA/Network_DATA/Rosters/PHYS1116_Fall_2020_roster.csv',
               csv_skip = [1, 2], merge_name = True, first_name_col = 'Q5c', last_name_col = 'Q5b')

cols_Fall2020POST2213 = ['Q5a', 'Q5b', 'Q5c', 'Q222', 'Q218', 'Q220.1', 'Q216']
CreateEdgelist('C:/Users/Cole/Documents/DATA/Network_DATA/Fall2020_POST/Fall2020_Cornell_University_Phys_2213_Holmes_POST_R_ZfQFj15tAmokZR7_January+10,+2021_17.39.csv', 
               cols_Fall2020POST2213, 
               roster_file = 'C:/Users/Cole/Documents/DATA/Network_DATA/Rosters/PHYS2213_Fall_2020_roster.csv',
               csv_skip = [1, 2], merge_name = True, first_name_col = 'Q5c', last_name_col = 'Q5b')

CreateEdgelist('C:/Users/Cole/Documents/DATA/Network_DATA/Fall2020_POST/Fall2020_Cornell_University_Phys_2218_Holmes_POST_R_2723pk178fyXrG5_January+10,+2021_17.40.csv', 
               cols_Fall2020POST1116_2218, 
               roster_file = 'C:/Users/Cole/Documents/DATA/Network_DATA/Rosters/PHYS2218_Fall_2020_roster.csv',
               csv_skip = [1, 2], merge_name = True, first_name_col = 'Q5c', last_name_col = 'Q5b')

In [29]:
pd.read_csv('C:/Users/Cole/Documents/DATA/Network_DATA/Fall2020_POST/Fall2020_Cornell_University_Phys_2218_Holmes_POST_R_2723pk178fyXrG5_January+10,+2021_17.40.csv').columns[1:100]

Index(['EndDate', 'Status', 'IPAddress', 'Progress', 'Duration (in seconds)',
       'Finished', 'RecordedDate', 'ResponseId', 'RecipientLastName',
       'RecipientFirstName', 'RecipientEmail', 'ExternalReference',
       'LocationLatitude', 'LocationLongitude', 'DistributionChannel',
       'UserLanguage', 'QID172040033', 'Q373', 'Q373_10_TEXT', 'Q373_11_TEXT',
       'Q373_9_TEXT', 'Q374', 'Q374_2_TEXT', 'Q375_2', 'Q375_1', 'Q375_3',
       'Q375_5', 'Q375_4', 'Q218_1', 'Q218_2', 'Q218_3', 'Q218_4', 'Q218_5',
       'Q376', 'Q376_4_TEXT', 'Q220', 'Q220_4_TEXT', 'Q377', 'Q377_4_TEXT',
       'Q222', 'Q222_4_TEXT', 'Q378', 'Q363_1', 'Q363_2', 'Q363_3', 'Q363_4',
       'Q364_First Click', 'Q364_Last Click', 'Q364_Page Submit',
       'Q364_Click Count', 'Q365_1', 'Q365_2', 'Q365_3', 'Q365_4', 'Q365_5',
       'Q365_6', 'Q365_7', 'Q365_8', 'Q365_9', 'Q365_10', 'Q366_First Click',
       'Q366_Last Click', 'Q366_Page Submit', 'Q366_Click Count', 'Q367_1',
       'Q367_2', 'Q367_3', 'Q36